## RaceCar NLP

In [ ]:
using JuMP
using NLopt
using Ipopt
using SplineUtils
using SplineRaceWay

In [ ]:
Pts = 40*[0 -1 -2 -3 -3.5 -3 -2 -1 -0.5 -1 -2 -3 -4 -5 -5.5 -5 -4.5 -5 -5 -4 -3 -2 -1 -1 -1 0 1 1 1 2 3 4 5 5 5 5 5 5 5 4 3 3 3 3 2 1 0; 
       0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -3 -2 -1 0 0 0 0]

degree = 3 # degree of spline
num_points = 10001
num_samples = 420
lane_width = 20.0
track = Raceway(Pts,degree,num_points,num_samples,lane_width)
track;

In [ ]:
### Racecar model

car_length = 4.8
car_width = 2.5
L = 4.5 # wheel base of vehicle
L₁ = 2 # distance from car center of mass to rear axle
L₂ = L - L₁ # distance from center of mass to front axle

# X = [s, n, ζ, u, v, ω] # state vector
# U = [u̇, δ] # control vector

# Bicycle geometry
# v = u*tan(δ)*(L₁/L)
# ω = u*tan(δ)/L
# u̇ = command
# δ = command

# Curvilinear Representation - s,n,θ
# dx = ds*cos(θ) # 1
# dy = ds*sin(θ) # 2
# dy_dx = tan(θ) # 3
# C = dθ_ds = atan2(dy,dx) # 4

# ψ = ζ + θ
# ṡ = (u*cos(ζ) - v*sin(ζ))/(1 - n*C)
# ṅ = (u*sin(ζ) + v*cos(ζ))
# ζ̇ = ω - C*ṡ
# dt = Sf(s)ds
# Sf = (1 - n*C)/(u*cos(ζ) - v*sin(ζ)) # inverse of ds/dt

# dn_ds = Sf*(u*cos(ζ) + v*sin(ζ))
# dζ_ds = Sf*ω - C
# du_ds = Sf(s)*u̇
# dv_ds = Sf(s)*v̇
# dω_ds = Sf(s)*ω̇

# f constraints: approximate derivatives with trapezoidal rule
# dx_ds = f()
# dn_ds -  Sf*(u*cos(ζ) + v*sin(ζ)) = 0

# S = track.s[1:N]
# T1 = zeros(N-1,N)
# T1[:,1:end-1] -= eye(N-1)
# T1[:,2:end] += eye(N-1)
# T2 = .5*abs(T1)
# Sd = T1*S
# T = kron(diagm(Sd)*T2,eye(n_))
# C = ones(N)
# Sdiff = diff(track.s)


In [ ]:
N = 20 # horizon length
m = 3 # dimensionality of control vector
n_ = 6 # dimensionality of state vector

car_length = 4.8
car_width = 2.5
L = 4.5 # wheel base of vehicle
L₁ = 2 # distance from car center of mass to rear axle
L₂ = L - L₁ # distance from center of mass to front axle

# optimizer = Model(solver=NLoptSolver(algorithm=:LD_MMA))
optimizer = Model(solver=IpoptSolver())
# parameters
@NLparameter(optimizer, S[i=1:N] == track.s[i])
@NLparameter(optimizer, C[i=1:N] == track.k[i])
@NLparameter(optimizer, Sd[i=1:N] == track.s[i+1] - track.s[i])
# for i in 1:N
#     setvalue(S[i], track.s[i])
#     setvalue(C[i], track.k[i])
# end
# for i in 1:N-1
#     setvalue(Sd[i], track.s[i+1] - track.s[i])
# end

# control vector
@variable(optimizer, u̇[1:N], start = 1.0)
@variable(optimizer, δ̇[1:N], start = 1.0)

# state vector
@variable(optimizer, n[1:N], start = 1.0)
@variable(optimizer, ζ[1:N], start = 1.0)
@variable(optimizer, u[1:N], start = 1.0)
@variable(optimizer, δ[1:N], start = 1.0)
@variable(optimizer, v[1:N], start = 1.0)
@variable(optimizer, v̇[1:N], start = 1.0)
@variable(optimizer, ω[1:N], start = 1.0)
@variable(optimizer, ω̇[1:N], start = 1.0)

# time scale factor: inverse of ds_dt
@NLexpression(optimizer, Sf[i=1:N], (1 - n[i]*C[i])/(u[i]*cos(ζ[i]) - v[i]*sin(ζ[i])))
# @variable(optimizer, Sf[1:N])
# for i in 1:N
#     @NLconstraint(optimizer, Sf[i] == (1 - n[i]*C[i])/(u[i]*cos(ζ[i]) - v[i]*sin(ζ[i]))) # set value of Sf
# end

# L objective
@NLobjective(optimizer, Min, sum(0.5*Sd[i]*(Sf[i+1] - Sf[i]) for i = 1:N-1))
# @NLobjective(optimizer, Min, u[N-5])

In [ ]:
getvalue(Sf);

In [ ]:
# constraints
μk = 1000.0 # normalized friction coefficient
δ_MAX = Float64(π)/4
δ̇_MAX = 24*Float64(π) # 
u_MAX = 100
u_MIN = 1
u̇_MAX = 1
u̇_MIN = -4

g = 9.81 # gravity force
n_MAX = 20.0 # half-width of track

# in
n₀ = 0 # n[2]
u₀ = 0 # u[2]
ω₀ = 0 # ω[2]
ζ₀ = 0 # ζ[2]

n₁ = 0
u₁ = 10 # some value
ω₁ = 0 # some value
ζ₁ = 0 # some value

# BOUNDARY CONSTRAINTS
# initial conditions
@NLconstraint(optimizer, n[1] == n₀)
@NLconstraint(optimizer, u[1] == u₀)
@NLconstraint(optimizer, ω[1] == ω₀)
@NLconstraint(optimizer, ζ[1] == ζ₀)
# final conditions
@NLconstraint(optimizer, n[N] == n₁)
@NLconstraint(optimizer, u[N] == u₁)
@NLconstraint(optimizer, ω[N] == ω₁)
@NLconstraint(optimizer, ζ[N] == ζ₁)

for i in 1:N-1
    # Derivative constraints
    @NLconstraint(optimizer, Sf[i] == (1 - n[i]*C[i])/(u[i]*cos(ζ[i]) - v[i]*sin(ζ[i]))) # set value of Sf
    @NLconstraint(optimizer, n[i+1]-n[i] == 0.5*Sd[i]*(Sf[i+1]*(u[i+1]*sin(ζ[i+1]) + v[i+1]*cos(ζ[i+1])) + 
    Sf[i]*(u[i]*sin(ζ[i]) + v[i]*cos(ζ[i]))))
    @NLconstraint(optimizer, ζ[i+1]-ζ[i] == 0.5*Sd[i]*(Sf[i+1]*(ω[i+1] - C[i+1]) + Sf[i]*(ω[i] - C[i])))
    @NLconstraint(optimizer, u[i+1]-u[i] == 0.5*Sd[i]*(Sf[i+1]*u̇[i+1] + Sf[i]*u̇[i]))
    @NLconstraint(optimizer, ω[i] == u[i]*tan(δ[i])*L₁/L) # ω is a function of u,δ
    @NLconstraint(optimizer, ω̇[i] == u̇[i]*tan(δ[i])*L₁/L + u[i]*(sec(δ[i])^2)*δ̇[i])
    @NLconstraint(optimizer, v[i] == ω[i]*L₁) # ω is a function of u,δ
    @NLconstraint(optimizer, v̇[i] == ω̇[i]*L₁) # ω is a function of u,δ
    @NLconstraint(optimizer, ω[i+1]-ω[i] == 0.5*Sd[i]*(Sf[i+1]*ω̇[i+1] + Sf[i]*ω̇[i]))
    @NLconstraint(optimizer, v[i+1]-v[i] == 0.5*Sd[i]*(Sf[i+1]*v̇[i+1] + Sf[i]*v̇[i]))
    
    # other constraints
    @NLconstraint(optimizer, u_MIN <= u[i] <= u_MAX) # max longitudinal velocity
    @NLconstraint(optimizer, u̇_MIN <= u̇[i] <= u̇_MAX) # max longitudinal acceleration
    @NLconstraint(optimizer, -δ_MAX <= δ[i] <= δ_MAX) # max turning angle
    @NLconstraint(optimizer, -δ̇_MAX <= δ̇[i] <= δ̇_MAX) # derivative
    @NLconstraint(optimizer, (u[i]*u[i] + v[i]*v[i])*C[i] <= μk*g) # cornering speed
    @NLconstraint(optimizer, -n_MAX <= n[i] <= n_MAX) # half width of track
    
#     @NLconstraint(optimizer, v[i] == 0)
#     @NLconstraint(optimizer, ζ[i] == 0)
#     @NLconstraint(optimizer, n[i] == 0)
#     @NLconstraint(optimizer, u[i+1] - u[i] == u̇[i])
end

In [ ]:
status = solve(optimizer)

In [ ]:
N = 20

S = track.s[1:N]
T1 = zeros(N-1,N)
T1[:,1:end-1] -= eye(N-1)
T1[:,2:end] += eye(N-1)
Sd = T1*S
C = track.k[1:N]
# T2 = T1*.5
# L = ones(N,1)
# Sd'*T2*L
C;

In [ ]:
# L objective
@NLobjective(optimizer, Min, sum(0.5*Sd[i]*((1 - n[i+1]*C[i+1])/(u[i+1]*cos(ζ[i+1]) - v[i+1]*sin(ζ[i+1])) - 
    (1 - n[i]*C[i])/(u[i]*cos(ζ[i]) - v[i]*sin(ζ[i]))) for i = 1:N-1))